In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
from sklearn.model_selection import train_test_split

## The data

The data is from electron microscopy - images of grpahene sheets. The typical image is $(256, 256)$ but we would like to take small patches of this and identify - is there a defect in the patch.

Lets go through a few of those things. Below we will look at 

* A full image
* A patch of an image with no defects
* A patch with a defect

### Full image

In [ ]:
graphene = np.load('full-stack.npy') ## This has 180 different microgrpahs each (256, 256)
sample = np.squeeze(graphene[54])

In [ ]:
print(sample.shape)

In [ ]:
plt.imshow(sample)
plt.colorbar()

#### Description
The blue (high electron density) correspnds to atoms and the green correspnds to background.

You can see that in the middle of the sheet most of it is a pretty regular array of hexagons, this is what we exepct for the perfect lattice. You will also hopefully see some spots where the hexagon is broken. This many of these are the result of missing atom defects. 

Notice the edges look a bit weird too. This is standard, and we will generally ignore the edges when analysing.



### Perfect patch

There is a training dataset of perfect patches 

In [ ]:
pp = np.load('./perfect_patches.npy')
perfect_sample = pp[0] 
print(pp.shape)

In [ ]:
n1 = np.sum(perfect_sample, axis=0)
plt.plot(n1)
e1 = np.std(n1)/np.mean(n1)
n2 = np.sum(perfect_sample, axis=1)
e2 = np.std(n2)/np.mean(n2)
plt.plot(n2)
e1, e2, np.mean(n1), np.mean(n2), np.std(n1), np.std(n2)


In [ ]:
plt.imshow(perfect_sample)

### Defect patch

There is also a set of defective patches. Note - these are for testing only, not for training. I guess we could imagine training a classifier on these and the perfect patches - but that would be no fun :)

In [ ]:
dp = np.load('./defect_patches.npy')
defect_sample = dp[0] 
print(dp.shape)


In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(defect_sample)

In [ ]:
n1 = np.sum(defect_sample, axis=0)
plt.plot(n1)
e1 = np.std(n1)/np.mean(n1)
n2 = np.sum(defect_sample, axis=1)
e2 = np.std(n2)/np.mean(n2)
plt.plot(n2)
e1, e2, np.mean(n1), np.mean(n2), np.std(n1), np.std(n2)


Get patches perfect and defect, and put them randomly in a list, with the ground truth saved

In [ ]:
patches = np.concatenate([pp, dp])
gt = np.concatenate([np.zeros(len(pp)), np.ones(len(dp))])
xy = list(zip(patches, gt)) # put labels with data in (data, label) tuple
random.shuffle(xy) # shuffle the tuples

In [ ]:
patches, gt = list(zip(*xy)) # unzip the tuples to get patches and ground truths as vectors

Splitting into training and testing

# Helper functions
The below fonctions help with the processing.

#### Model helper functions

In [ ]:
def std(image) -> tuple:
    """Returns the standard deviations along the x and y axes.

    Args:
        image (Array): image to compute its standard deviations.

    Returns:
        tuple: (0-axis std, 1-axis std).
    """
    image1 = np.sum(image,axis=1)
    image0 = np.sum(image,axis=0)
    stdim0 = np.std(image0)
    stdim1 = np.std(image1)
    return (stdim0,stdim1)

### Image helper functions

In [ ]:
def flip_horizontally(imgs):
    flipped = np.array([cv2.flip(img, 0) for img in imgs])
    return flipped

def flip_vertically(imgs):
    flipped = np.array([cv2.flip(img, 1)  for img in imgs])
    return flipped

def blur(imgs, kernel = (5,5)):
    blurred = np.array([cv2.blur(img, kernel) for img in imgs])
    return blurred

def augment(defect_imgs):
    """Augments the dataset by applying a series of transformations.

    Args:
        defect_imgs (np.array): array of images to augment.

    Returns:
        np.array: augmented array.
    """
    # Let's first apply some initial transformations.
    blurred = blur(defect_imgs)
    flip_h = flip_horizontally(defect_imgs)
    flip_v = flip_vertically(defect_imgs)
    flip_h_v = flip_horizontally(flip_v)
    b_h = blur(flip_h)
    b_v = blur(flip_v)
    b_h_v = blur(flip_h_v)
    data = [
        defect_imgs,
        blurred,
        flip_h,
        flip_v,
        flip_h_v,
        b_h,
        b_v,
        b_h_v,
            ]
    
    # Let's now apply mathematical transformations to the newly 
    # augmented data set.
    cosed = []
    sined = []
    taned = []
    exped = []
    
    for sub_data in data:
        cosed.append(np.cos(sub_data))
        sined.append(np.sin(sub_data))
        taned.append(np.tan(sub_data))
        exped.append(np.exp(sub_data))
        
    data = np.concatenate((*data, *cosed, *sined, *taned, *exped), axis=0)
    data = np.concatenate((data, blur(data, kernel=[7,7])), axis=0)
    
    return data

# Data Augmentation
TODO: Here we augment the defect images to have a 50-50 ratio between the perfect and defect images.

In [ ]:
limit = 2000
X_defect = augment(dp)[:limit]

# Data Splitting
Here we are splitting the data into a:
- training data set,
- testing data set.

In [ ]:
# Perfect patches
X_perfect = pp[:limit]
y_perfect = np.zeros(len(X_perfect))
X_perfect_train, X_perfect_test, y_perfect_train, y_perfect_test = train_test_split(
    X_perfect,
    y_perfect,
    test_size=0.20,
    random_state=42,)

In [ ]:
# Defect patches
y_defect = np.ones(len(X_defect))
X_defect_train, X_defect_test, y_defect_train, y_defect_test = train_test_split(
    X_defect, 
    y_defect, 
    test_size=0.20, 
    random_state=42,)

In [ ]:
# Concatenate the perfect and defect patches into one training and testing data sets.
X_train = np.concatenate((X_perfect_train, X_defect_train), axis = 0)
y_train = np.concatenate((y_perfect_train, y_defect_train), axis = 0)
print(f"X_train :\t {X_train.shape} | y_train :\t {y_train.shape}")

X_test = np.concatenate((X_perfect_test, X_defect_test), axis = 0)
y_test = np.concatenate((y_perfect_test, y_defect_test), axis = 0)
print(f"X_test :\t {X_test.shape} | y_test :\t {y_test.shape}")

In [ ]:
ppL = []
dpL = []
for _, img in enumerate(X_perfect_train):
    ppL.append(std(img))
for _, img in enumerate(X_defect_train):
    dpL.append(std(img))


#### Plot stds for perfect images

In [ ]:
ph1 = []
ph0 = []
for std0, std1 in ppL:
    ph1.append(std1)
    ph0.append(std0)

fig, axes = plt.subplots(nrows=1, ncols=2)
axes[0].plot(range(len(ppL)), ph0)
axes[1].plot(range(len(ppL)), ph1)
fig.tight_layout()

In [ ]:
dh1 = []
dh0 = []
for std0, std1 in dpL:
    dh1.append(std1)
    dh0.append(std0)

fig, axes = plt.subplots(nrows=1, ncols=2)
axes[0].plot(range(len(dpL)), dh0)
axes[1].plot(range(len(dpL)), dh1)
fig.tight_layout()

### Transformt the input training data

In [ ]:
flattens= []
for input in X_train:
    stds = std(input)
    flattened = np.ndarray.flatten(input)
    flattened = np.append(flattened, stds)
    
    flattens.append(flattened)

flattens = np.array(flattens)
print(flattens.shape)
